While technology offers incredible resources for learning and exploration, it's vital to remember the importance of approaching our studies with honesty and effort. To maintain the integrity of our academic community, we kindly ask that you refrain from using ChatGPT or similar AI tools to directly generate solutions for your homework assignments.

Instead, we encourage you to use technology responsibly as a supplement to your learning—leveraging it for research, supplementary understanding, and exploration, rather than as a means to bypass the hard work that learning often requires.


Please be aware that we will be implementing tools and measures to check submissions for their originality and to ensure they are the result of your own effort and understanding.


## Problem Statement

To solve the linear system $Ax = b$, where $A$ is nonsingular, we consider a projection method that uses a two-dimensional space at each step. At a given step, we define $K = \text{Span}\{r, Ar\}$, where $r = b - Ax$ is the current residual, and $L = AK$.

### Tasks

1. For a basis of $K$, we use the vectors $p_1 = \frac{r}{\|Ar\|_2}$ and the vector $p_2 = Ap_1 - \gamma p_1$ such that $Ap_2$ is orthogonal to $Ap_1$. **Give the formula for computing $p_2$.**

From the definition of $p_2$, we can seee that $\mathbf{A}p_2 = \mathbf{A}^2p_1 - \gamma\mathbf{A}p_1$. Now using the definition of orthogonality, we compute
\begin{align}
\langle \mathbf{A}p_1, \mathbf{A}p_2\rangle &= p_1^\top\mathbf{A}^\top \left(\mathbf{A}^2p_1 - \gamma\mathbf{A}p_1\right)\\
&= p_1^\top\mathbf{A}^\top\mathbf{A}^2p_1 - \gamma p_1^\top\mathbf{A}^\top\mathbf{A}p_1\\
&= 0,
\end{align}
where we enforce the orthogonality of the inner product. Observe that both terms start with a row vector and end with a column vector, which means both are scalars. This means that we can manipulate the equation like scalar algebra to solve for $\gamma$, to find
\begin{align}
\gamma p_1^\top\mathbf{A}^\top\mathbf{A}p_1 &= p_1^\top\mathbf{A}^\top\mathbf{A}^2p_1\\
\gamma &= \frac{p_1^\top\mathbf{A}^\top\mathbf{A}^2p_1}{p_1^\top\mathbf{A}^\top\mathbf{A}p_1}.
\end{align}

Lastly, we use these facts to write an efficient code for computing $p_2$.

```
alpha = A * p1
gamma = ((alpha.T * A) * alpha)/ norm(alpha) ** 2
p2 = alpha - gamma * p1
```


2. **Write the algorithm** for performing the projection method described above.

First it is helpful to remind ourselves the basics of projection methods. $\mathbf{L}$ is the constraint space and $\mathbf{K}$ is the search space, where since $\mathbf{A}$ is nonsingular and not SPD, we have the relationship $\mathbf{L} = \mathbf{AK}$. In terms of subspaces, $\mathbf{V}$ is a subspace of the search space, and $\mathbf{W}$ is a subspace of the constraint space. We are told that the basis of $\mathbf{K}$ are the vectors $p_i$. And since we know that $\mathbf{L} = \mathbf{AK}$, we get:

\begin{align}
\mathbf{V} &= \begin{bmatrix} \vec{p}_1 & \vec{p}_2 \end{bmatrix}\\
\mathbf{W} &= \begin{bmatrix} \mathbf{A}\vec{p}_1 & \mathbf{A}\vec{p}_2 \end{bmatrix}
\end{align}

Now using the fact that $\dim \mathbf{L} = \dim \mathbf{W} = 2$, we can write the update step from the generic projection formula
\begin{align}
\vec{x}_2 &= \vec{x}_0 + \mathbf{V}\left(\mathbf{W}^\top \mathbf{AV}\right)^{-1}\mathbf{W}^\top \vec{r}_0\\
&= \vec{x}_0 + \mathbf{V}\left(\left(\mathbf{AV}\right)^\top \mathbf{AV}\right)^{-1}\left(\mathbf{AV}\right)^\top \vec{r}_0\\
&= \vec{x}_0 + \mathbf{V}\mathbf{V}^{-1}\mathbf{A}^{-1} \mathbf{A}^{-\top}\mathbf{V}^{-\top}\mathbf{V}^\top\mathbf{A}^\top \vec{r}_0\\
&= \vec{x}_0 + \left(\mathbf{A}^{\top}\mathbf{A}\right)^{-1}\mathbf{A}^\top\vec{r}_0\\
&= \vec{x}_0 + \mathbf{A}^{\dagger}\vec{r}_0
\end{align}

3. **Analyze its convergence** and discuss to which other method this algorithm is mathematically equivalent.

This appears to be the Generalized Conjugate Residual (GCR) Method. Since we have $K_2(\vec{r}, \mathbf{A})$, which is a 2 dimensional Krylov subspace, this is restarting every two iterations, so it is actually GCR$(2)$, which is mathematically equivalent to GMRES(2); however, GCR(2) has to store both the $\vec{p}_i, \mathbf{A}\vec{p}_i, \forall i = 1,2$. Additionally this method seems to have more arithmetic operations than GMRES. 

In terms of convergence, we do expect that this method is better than just restarting after one iteration like MINRES, which only considers the 1D projection method. Looking ahead at Krylov 3, we can see that the convergence theory for GMRES for a generic nonsingular $A$.
\begin{align}
\frac{\|r_k\|}{\|b\|} &\leq \kappa_2(V)\inf_{p\in P_n} \max_{\lambda\in \sigma(A)} |p(A)|,
\end{align}
where $P_n$ denotes the set of polynomials of degree at most $n$ with $p(0)=1$, and $V$ is the matrix in the spectral decomposition, $A = V\Lambda V^T$.

Again, since GMRES and GCR are mathematically equivalent, we expect the convergence rate to be the same, but other considerations like storage and cost to compute the next iterate can affect the time per iteration.



